In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset, random_split
import glob
import pathlib
from PIL import Image
import numpy as np
from tqdm import tqdm
import timm

/home/cyliu/anaconda3/envs/pair/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_label(path, labels):
    '''Gets label from the path'''
    label_str = path.parent.stem
    label = None

    if label_str == labels[0]:
        label = np.array([0], dtype=np.double)
    elif label_str == labels[1]:
        label = np.array([1], dtype=np.double)
    return label

In [3]:
class PairDataset(Dataset):
    '''Provides an object for the MRI data that can be iterated.'''
    def __init__(self, root_dir, labels, transform=None):

        self.root_dir = root_dir  # root_dir="../data/"
        self.transform = transform
        self.directories = []
        self.len = 0
        self.labels = labels


    
        train_dirs = []

        for label in labels:
            train_dirs.append(root_dir + label)


        for train_dir in train_dirs:
            for path in glob.glob(train_dir + "/*.jpg"):
                self.directories.append(pathlib.Path(path))

        self.len = len(self.directories)

    def __len__(self):

        return self.len

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        repeat = True

        while repeat:
            try:
                path = self.directories[idx]
                image = Image.open(path).convert('RGB')
                label = get_label(path, self.labels)
        
                if self.transform:
                    image = self.transform(image)

                return image, label

            except IndexError as index_e:
                print(index_e)
                if idx < self.len:
                    idx += 1
                else:
                    idx = 0

        return image, label

In [19]:
# Define paths
data_dir = '/home/cyliu/dataset/paired/green/'


# Define transformations for the training and validation sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Split the dataset into training and validation sets

# Create separate datasets for training and validation
train_dataset = PairDataset(root_dir=data_dir, labels=["stain", "no stain"], transform=data_transforms['train'])
val_dataset = PairDataset(root_dir=data_dir, labels=["stain", "no stain"], transform=data_transforms['val'])

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, _ = random_split(train_dataset, [train_size, val_size])
_, val_dataset = random_split(val_dataset, [train_size, val_size])

# Create data loaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
}
dataset_size = {'train': train_size, 'val': val_size}

In [20]:
# Check if CUDA is available
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Load a pre-trained ResNet50 model
# model = models.resnet50(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)


model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=2)
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=2)
#model = timm.create_model('convnextv2_base', pretrained=True, num_classes=2)
# Modify the final layer to match the number of classes (2 in this case)


# Move the model to the appropriate device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=5e-4)

In [9]:
def train_model(model, criterion, optimizer, dataset_size, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device).squeeze(1).long()

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
            
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects.double() / dataset_size[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        print()

    print(f'Best val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [72]:
# resnet50
model = train_model(model, criterion, optimizer, dataset_size, num_epochs=100)

Epoch 0/99
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:20<00:00,  1.31s/it]


train Loss: 0.8060 Acc: 0.5510


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.53s/it]


val Loss: 3.9586 Acc: 0.5360

Epoch 1/99
----------


...

val Loss: 0.6420 Acc: 0.6960

Epoch 99/99
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:20<00:00,  1.29s/it]


train Loss: 0.4728 Acc: 0.7550


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.52s/it]

val Loss: 0.6750 Acc: 0.7120

Best val Acc: 0.800000


In [77]:
model = train_model(model, criterion, optimizer, dataset_size, num_epochs=100)

Epoch 0/99
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:25<00:00,  1.61s/it]


train Loss: 2.5356 Acc: 0.5700


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.83s/it]


val Loss: 2.5949 Acc: 0.6280

Epoch 1/99
----------


...

val Loss: 0.1076 Acc: 0.9560

Epoch 99/99
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:25<00:00,  1.60s/it]


train Loss: 0.1711 Acc: 0.9280


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.83s/it]

val Loss: 0.1604 Acc: 0.9400

Best val Acc: 0.960000


In [13]:
model = train_model(model, criterion, optimizer, dataset_size, num_epochs=100)

Epoch 0/99
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:21<00:00,  1.33s/it]


train Loss: 2.4018 Acc: 0.5660


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.91s/it]


val Loss: 1.4184 Acc: 0.6160

Epoch 1/99
----------


...

val Loss: 0.3618 Acc: 0.9560

Epoch 99/99
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:26<00:00,  1.63s/it]


train Loss: 0.0881 Acc: 0.9610


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.90s/it]

val Loss: 0.3604 Acc: 0.9560

Best val Acc: 0.972000


In [21]:
# efficientb4 green
model = train_model(model, criterion, optimizer, dataset_size, num_epochs=100)

Epoch 0/99
----------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:14<00:00,  1.33s/it]


train Loss: 1.4345 Acc: 0.5991


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.97s/it]


val Loss: 1.1663 Acc: 0.6727

Epoch 1/99
----------


...

val Loss: 0.1808 Acc: 0.9697

Epoch 99/99
----------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:14<00:00,  1.29s/it]


train Loss: 0.0549 Acc: 0.9787


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.88s/it]


val Loss: 0.1648 Acc: 0.9636

Best val Acc: 0.981818


In [22]:
torch.save(model, "efficientnet_b4_green.pth")

In [85]:
model_names = timm.list_models(pretrained=False)

In [81]:
print(model_names)

['bat_resnext26ts.ch_in1k', 'beit_base_patch16_224.in22k_ft_in22k', 'beit_base_patch16_224.in22k_ft_in22k_in1k', 'beit_base_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_224.in22k_ft_in22k', 'beit_large_patch16_224.in22k_ft_in22k_in1k', 'beit_large_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_512.in22k_ft_in22k_in1k', 'beitv2_base_patch16_224.in1k_ft_in1k', 'beitv2_base_patch16_224.in1k_ft_in22k', 'beitv2_base_patch16_224.in1k_ft_in22k_in1k', 'beitv2_large_patch16_224.in1k_ft_in1k', 'beitv2_large_patch16_224.in1k_ft_in22k', 'beitv2_large_patch16_224.in1k_ft_in22k_in1k', 'botnet26t_256.c1_in1k', 'caformer_b36.sail_in1k', 'caformer_b36.sail_in1k_384', 'caformer_b36.sail_in22k', 'caformer_b36.sail_in22k_ft_in1k', 'caformer_b36.sail_in22k_ft_in1k_384', 'caformer_m36.sail_in1k', 'caformer_m36.sail_in1k_384', 'caformer_m36.sail_in22k', 'caformer_m36.sail_in22k_ft_in1k', 'caformer_m36.sail_in22k_ft_in1k_384', 'caformer_s18.sail_in1k', 'caformer_s18.sail_in1k_384', 'caformer_s

In [86]:
print(model_names)

['bat_resnext26ts', 'beit_base_patch16_224', 'beit_base_patch16_384', 'beit_large_patch16_224', 'beit_large_patch16_384', 'beit_large_patch16_512', 'beitv2_base_patch16_224', 'beitv2_large_patch16_224', 'botnet26t_256', 'botnet50ts_256', 'caformer_b36', 'caformer_m36', 'caformer_s18', 'caformer_s36', 'cait_m36_384', 'cait_m48_448', 'cait_s24_224', 'cait_s24_384', 'cait_s36_384', 'cait_xs24_384', 'cait_xxs24_224', 'cait_xxs24_384', 'cait_xxs36_224', 'cait_xxs36_384', 'coat_lite_medium', 'coat_lite_medium_384', 'coat_lite_mini', 'coat_lite_small', 'coat_lite_tiny', 'coat_mini', 'coat_small', 'coat_tiny', 'coatnet_0_224', 'coatnet_0_rw_224', 'coatnet_1_224', 'coatnet_1_rw_224', 'coatnet_2_224', 'coatnet_2_rw_224', 'coatnet_3_224', 'coatnet_3_rw_224', 'coatnet_4_224', 'coatnet_5_224', 'coatnet_bn_0_rw_224', 'coatnet_nano_cc_224', 'coatnet_nano_rw_224', 'coatnet_pico_rw_224', 'coatnet_rmlp_0_rw_224', 'coatnet_rmlp_1_rw2_224', 'coatnet_rmlp_1_rw_224', 'coatnet_rmlp_2_rw_224', 'coatnet_rmlp_2